In [3]:
from astropy import units as u
from sedfitter import fit
from sedfitter.extinction import Extinction
import os

In [4]:
# path to models
model_dir = '/Users/yaolun/data/models_r06/'

# Read in extinction
dust_path = '/Users/yaolun/Google Drive/dust_model/Weingartner_Draine/kext_albedo_WD_MW_5.5A_30_D03.ext'
extinction = Extinction.from_file(dust_path, columns=[0,1], wav_unit=u.micron, chi_unit=u.cm**2/u.g)

In [15]:
# set up the filters as the extraction of spectro-photometric fluxes
def filter_hyperion(wave):
    from sedfitter.filter import Filter
    import numpy as np
    import astropy.constants as const
    c = const.c.cgs.value
    
    if (wave < 50.) & (wave >= 5.):
        res = 60.
    elif wave < 5.:
        res = 10.
    else:
        res = 1000.
    
    wave_min = wave*(1-1./res)
    wave_max = wave*(1+1./res)
    
    if wave not in [70, 100, 160, 250, 350, 500]:
        f = Filter()
        f.name = str(wave)+'um'
        f.central_wavelength = wave*u.micron
        f.nu = (np.linspace(wave_min, wave_max, 100) * u.micron).to(u.Hz, equivalencies=u.spectral())
        f.response = np.ones_like(f.nu.value)
        f.normalize()
    # herschel filters
    else:
        import sys
        sys.path.append('/Users/yaolun/programs/misc/hyperion/')
        from phot_filter import phot_filter
        
        if wave == 70:
            fil_name = 'Herschel PACS 70um'
            name = 'P1'
        elif wave == 100:
            fil_name = 'Herschel PACS 100um'
            name = 'P2'
        elif wave == 160:
            fil_name = 'Herschel PACS 160um'
            name = 'P3'
        elif wave == 250:
            fil_name = 'Herschel SPIRE 250um'
            name = 'S1'
        elif wave == 350:
            fil_name = 'Herschel SPIRE 350um'
            name = 'S2'
        elif wave == 500:
            fil_name = 'Herschel SPIRE 500um'
            name = 'S3'
        
        print fil_name
        
        filter_func = phot_filter(fil_name, '/Users/yaolun/misc/hyperion/')
        
        f = Filter()
        f.name = name
        f.central_wavelength = wave * u.micron
        f.nu = (filter_func['wave'] * u.micron).to(u.Hz, equivalencies=u.spectral())
        f.response = filter_func['transmission']
    return f

In [11]:
# monochromatic convolution on models
from sedfitter import convolve
mono_wav = [8.5, 9, 9.7, 10, 10.5, 11, 16, 20, 35]
mono_filters = []

for iwav in mono_wav:
    filter_dum = filter_hyperion(iwav)
    if not os.path.exists(model_dir+'/convolved/'+filter_dum.name+'.fits'):
        mono_filters.append(filter_dum)

In [16]:
# Herschel filters convolution on models
herschel_wave = [70, 100, 160, 250, 350, 500]
herschel_filters = []

for iwav in herschel_wave:
    filter_dum = filter_hyperion(iwav)
    if not os.path.exists(model_dir+'/convolved/'+filter_dum.name+'.fits'):
        herschel_filters.append(filter_dum)

Herschel PACS 70um


TypeError: phot_filter() takes exactly 1 argument (2 given)

In [12]:
convolve.convolve_model_dir(model_dir,mono_filters)

INFO: 200710 SEDs found in /Users/yaolun/data/models_r06/ [sedfitter.convolve.convolve]
INFO: Rebinning filters [sedfitter.convolve.convolve]


In [20]:
# Define filters and apertures
filters = ['I3','I4','8.5um','9um','9.7um','10um','10.5um','11um','16um','20um','M1','35um','']
apertures = [7.2, 7.2, 7.2] * u.arcsec

fit('/Users/yaolun/bhr71/best_calibrated/bhr71_data_for_sedfitter_test.txt', filters, apertures, model_dir,
    'outputfit.info', extinction_law=extinction, distance_range=[180, 220] * u.pc, av_range=[0, 10])

 ------------------------------------------------------------
  => Model parameters
 ------------------------------------------------------------

   Models              :  YSO models
   Log[d] stepping     :  0.01
   Number of distances :  9

 ------------------------------------------------------------
  => Reading in convolved fluxes
 ------------------------------------------------------------

   Reading /Users/yaolun/data/models_r06//convolved/I3.fits
   Reading /Users/yaolun/data/models_r06//convolved/I4.fits
   Reading /Users/yaolun/data/models_r06//convolved/8.5um.fits
 ------------------------------------------------------------
  => Fitting parameters
 ------------------------------------------------------------

   Minimum A_V      :     0.000 mag
   Maximum A_V      :    10.000 mag
   Minimum distance :   180.000 pc
   Maximum distance :   220.000 pc

 ------------------------------------------------------------
  => Output parameters
 -------------------------------------